In [1]:
import requests,time,datetime
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import utils.DateAndTimeUtil as datu
import utils.CommonUtil as cu
import utils.StockSpider as ss

In [14]:
df_hsA = pd.read_excel("data/沪深A股.xls",converters= {u'StockCode':str})

In [15]:
df = df_hsA

In [4]:
# dir_path = 'E:/wangtao/PythonWorkSpace/SpiderSpace/Stock_Companies/data/'
# ss.getAnnualReports(dir_path,df,'2017')

# ss.getStockDataByType('E:/wangtao/PythonWorkSpace/SpiderSpace/Stock_Companies/data/')
ss.getStockDataByType('E:/PythonWorkSpace/DataAnalysis/Stock_Companies/data/')


正在获取：沪深A股...请勿关闭浏览器
第2页-第19条...

Traceback (most recent call last):
  File "E:\PythonWorkSpace\DataAnalysis\Stock_Companies\utils\StockSpider.py", line 136, in getStockDataByType
    print('数据下载完毕，已保存到'+file_path)
UnboundLocalError: local variable 'file_path' referenced before assignment


In [17]:
# 显示所有列
pd.set_option('display.max_columns', None)
# 显示所有行
pd.set_option('display.max_rows', None)

In [18]:
df_2014 = pd.read_excel("data/2014年报.xls",converters= {u'stock_code':str})
df_2015 = pd.read_excel("data/2015年报.xls",converters= {u'stock_code':str})
df_2016 = pd.read_excel("data/2016年报.xls",converters= {u'stock_code':str})
df_2017 = pd.read_excel("data/2017年报.xls",converters= {u'stock_code':str})
df_2018 = pd.read_excel("data/2018年报.xls",converters= {u'stock_code':str})

In [19]:
code_list = []
reports = [df_2014,df_2015,df_2016,df_2017,df_2018]
for df_report in reports:
    codes = []
    for index, row in df_report.iterrows():
        # 净资产收益率大于15%
        if (row['stock_code'][0] != '3') & \
            (row['avg_roe'] > 0.15) & \
           (row['total_revenue_gr'] > 0) & \
            (row['basic_eps'] > 0) & \
            (row['np_per_share'] > 0) & \
            (row['operate_cash_flow_ps'] > 0):
#             (row['operate_cash_flow_ps_gr'] > 0):
            #满足条件的加入
            codes.append(row['stock_code'])
            
    code_list.append(codes)

In [20]:
intersection = code_list[0]
# 取每年的交集
for i in range(1,len(code_list)):
    intersection = list(set(intersection).intersection(set(code_list[i])))

In [21]:
df_target = df[df['StockCode'].apply(lambda x: (x in intersection))]

In [22]:
df_target.shape

(86, 19)

In [23]:
df_target = df_target[df_target['PE_D'].apply(lambda x:(x != '未公布') & (x != '0'))]
df_target = df_target[df_target['PE_S'].apply(lambda x:(x != '未公布') & (x != '0'))]
# 新增市盈率中位数
df_target['PE_M'] = df_target['StockCode'].apply(lambda x:ss.getPEMedian(x))
df_target = df_target[df_target['PE_M'] != 0]

df_target['pe_rate'] = df_target.apply(lambda x:(float(x['PE_D']) / x['PE_M']),axis=1)
df_target = df_target[df_target['pe_rate'] < 0.6]

In [24]:
df_target.shape

(5, 21)

In [25]:
df_target.sort_values(by=['pe_rate'],ascending=True)

,StockCode,StockName,Price,PriceLimit,QuantityRelativeR,TurnoverRate,Trade,PB,PE_S,PE_D,EarningsPerShare,NetProfitDes,ROE,GrossProfitRate,NetAssetValuePerShare,CapitalStock,ScaleShareType,FinanceAnalize,MoneyFlowPerShare,PE_M,pe_rate
1635,600566,济川药业,24.15,-0.29%,0.76,0.57%,医药生物 — 中药Ⅱ,3.45,11.66,11.306,1.60元,13.06亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降0.24%\t...,23.05%,84.45%,7.00元,8.15亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,"三线蓝筹,白马股,绩优股,",2.19元,27.660,0.408749
2099,002262,恩华药业,11.86,-0.67%,0.97,1.01%,医药生物 — 化学制药,3.48,23.04,17.412,0.52元,5.21亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长23.34%\t...,16.16%,60.53%,3.41元,10.19亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,"绩优股,",0.55元,41.105,0.423598
3673,000963,华东医药,22.21,-10.01%,6.16,5.06%,医药生物 — 化学制药,3.33,17.14,13.170,1.26元,22.13亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长22.33%\...,20.14%,32.34%,6.66元,17.50亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,"三线蓝筹,",0.52元,28.225,0.466608
1852,002572,索菲亚,20.98,-0.47%,0.67,1.53%,轻工制造 — 家用轻工,3.77,19.96,19.943,0.79元,7.20亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长4.05%\t\...,14.80%,37.25%,5.57元,9.12亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,"绩优股,",0.66元,39.520,0.504631
2198,603886,元祖股份,18.47,-0.75%,0.49,0.80%,食品饮料 — 食品加工,2.92,18.34,12.075,1.15元,2.75亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长6.94%\t\...,18.56%,65.91%,6.33元,2.40亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t小盘股\t\t\t\t\t\t\t\...,"绩优股,",2.64元,22.240,0.542941


In [57]:
df_target.shape

(98, 20)